# Q&A with Rag using Local Model

### Document Loading

First, install packages needed for local embeddings and vector storage.

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub gpt4all langchain-chroma pypdf

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Load and split an example document.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/Owner-Manual-KICKS-e-POWER-EN.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

Next, the below steps will download the GPT4All embeddings locally (if you don't already have them).

In [ ]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = GPT4AllEmbeddings(
    model_name="~/.cache/lm-studio/models/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf",
    gpt4all_kwargs={'allow_download': 'True'}
)

faiss_index = FAISS.from_documents(documents = docs, embedding = embeddings)
docs = faiss_index.similarity_search("What is LayoutParser?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])